# Tutorial 2 - Hierarchical Agents

## Functionalities:
- An agent (referred as meta agent) can also assign other agents (referred as inner agents) as the function call, thereby creating a hierarchical structure
- Whenever the inner agent is called, we will run the full inner agent's process until termination
- Maximum number of steps for each agent (meta agent or inner agent) can be defined in `max_subtasks` (default: 5) variable when initialising agent

## Additional Details:
- The inner agent will take in meta agent's instruction as input
- In addition, the inner agent will also have access to meta agent's task and subtasks completed for global context
- Inner agent's output will directly go into meta agent's subtasks completed dictionary, enabling it to directly update global context

## Use Agents Manually for greater control
- You can use agents manually using `use_agents(agent_name: str, agent_task: str)` The outputs of the agents will be stored into `subtasks_completed` of the meta agent automatically
- You can also interact with the meta agent using `reply_user(task: str, stateful: bool = False)`. This generates a reply for the agent to the `task` using `subtasks_completed` as context. If `stateful` is true, this will also store the output into `subtasks_completed`
- If `subtasks_completed` is too lengthy, you can run `summarise_subtasks_completed(task: str)` This summarises `subtasks_completed` according to what is relevant to the `task`

## Design Philosophy:
- Bottom layers can get context of the top layers for more context-specific generation
- Top layers choose bottom layers in a hierarchical fashion so as to limit the number of choices at each level for better performance
- `subtasks_completed` serves as the global context for the meta agents and the inner agents, and can be summarised as needed, e.g. when performing a new task to filter out irrelevant information 

# Install TaskGen

In [1]:
# Set up API key and do the necessary imports
import os
from taskgen import *

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Define Agents and Meta Agents

In [2]:
def random_number_from_string(input_string):
    ''' Returns a random number between 1 and 10 when given an input string '''
    import hashlib
    import random
    # Hash the input string
    hashed_string = hashlib.sha256(input_string.encode()).hexdigest()
    
    # Convert the hashed string to an integer
    hashed_integer = int(hashed_string, 16)
    
    # Seed the random number generator with the hashed integer
    random.seed(hashed_integer)
    
    # Generate a random number between 1 and 10
    return random.randint(1, 10)

In [3]:
# assign a dish pricing function for economist
def dish_price(list_of_dish_names: list) -> dict:
    ''' Takes in <list_of_dish_names> and outputs price '''
    if not isinstance(list_of_dish_names, list):
        list_of_food_items = [list_of_dish_names]
    output = []
    for each in list_of_dish_names:
        output.append('$'+str(random_number_from_string(each)))
    return output
    
fp = Function(fn_description = "Takes in <list_of_dish_names: list> and outputs price", 
              output_format = {"Price": "Dish name as key and price as value, type: list"}, 
              examples = {"list_of_dish_names": ["Pasta", "Pizza"], "Price": ['$5', '$2']},
              external_fn = dish_price)

In [4]:
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

In [5]:
# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [Agent('Chef', 'Takes in dish names and comes up with ingredients for all of htem at a go. Does not generate prices.'),
    Agent('Boss', 'Takes in menu items and curates them according to price'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with fictitious pricing for all of them at a go').assign_functions([fp])
    ]

In [6]:
my_agent.assign_agents(agent_list)

# Approach 1: Run a generic task with the Meta-Agent
- Let us run a general task to create 5 menu items for an Italian restaurant
- You can also specify how Agents interact with one another in the task, otherwise the Meta-Agent will infer what should be done

In [7]:
# my_agent.debug = False
my_agent.reset()
output = my_agent.run('Give me 5 menu items with name, description, ingredients and price based on Italian food choices.')

Subtask identified: Generate interesting dish names and descriptions using a Creative Writer
Calling function Creative Writer with parameters {'instruction': 'Generate interesting dish names and descriptions using a Creative Writer'}

### Start of Inner Agent: Creative Writer ###
Subtask identified: Brainstorm dish names and descriptions for various cuisine types
Getting LLM to perform the following task: Brainstorm dish names and descriptions for various cuisine types
> As a Creative Writer, I can generate interesting dish names and descriptions for various cuisine types. If you provide me with a specific cuisine, I can create unique menu items for that category.

Subtask identified: Create unique menu items for a specific cuisine type.
Getting LLM to perform the following task: Create unique menu items for a specific cuisine type.
> As a Creative Writer, I can generate interesting dish names and descriptions for various cuisine types. If you provide me with a specific cuisine, I can 

In [8]:
my_agent.reply_user()

'1. Name: Bella Risotto\n   Description: Creamy Arborio rice cooked with saffron, parmesan cheese, and a medley of seasonal vegetables.\n   Ingredients: Arborio rice, saffron, parmesan cheese, seasonal vegetables.\n   Price: $3\n\n2. Name: Truffle Tagliatelle\n   Description: Fresh homemade tagliatelle pasta tossed in a decadent truffle cream sauce with wild mushrooms.\n   Ingredients: Tagliatelle pasta, truffle cream sauce, wild mushrooms.\n   Price: $2\n\n3. Name: Sicilian Seafood Linguine\n   Description: Linguine pasta served with a rich tomato-based sauce, mixed seafood, capers, and fresh herbs.\n   Ingredients: Linguine pasta, tomato-based sauce, mixed seafood, capers, herbs.\n   Price: $5\n\n4. Name: Tuscan Chicken Marsala\n   Description: Tender chicken breast cooked in a Marsala wine sauce with mushrooms, served with roasted potatoes.\n   Ingredients: Chicken breast, Marsala wine sauce, mushrooms, roasted potatoes.\n   Price: $2\n\n5. Name: Limoncello Tiramisu\n   Description:

In [9]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Brainstorm dish names and descriptions for various cuisine types
As a Creative Writer, I can generate interesting dish names and descriptions for various cuisine types. If you provide me with a specific cuisine, I can create unique menu items for that category.

Subtask: Create unique menu items for a specific cuisine type.
As a Creative Writer, I can generate interesting dish names and descriptions for various cuisine types. If you provide me with a specific cuisine, I can create unique menu items for that category.

Subtask: Brainstorm dish names and descriptions for Italian cuisine.
1. Dish Name: Bella Risotto
Description: Cre

# Approach 2: Step through the process yourself as the Meta Agent
- You can run through your Agents one by one using the Meta Agent directly
- The outputs of the Inner Agents will be stored into `subtasks_completed` directly

In [23]:
# reset the state of the agent
my_agent.reset()
# currently should have no task and nothing in subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [24]:
# This will run the Agent Creative Writer (with shared subtasks_completed as meta agent)
# Output of task will go directly into subtasks_completed - we view the inner agent as simply an extension of the meta agent
my_agent.use_agent('Creative Writer', 'Generate 5 Italian dish names and descriptions')

Calling function Creative Writer with parameters {'instruction': 'Generate 5 Italian dish names and descriptions'}

### Start of Inner Agent: Creative Writer ###
Subtask identified: Research popular Italian dishes to gather inspiration for unique dish names and descriptions
Getting LLM to perform the following task: Research popular Italian dishes to gather inspiration for unique dish names and descriptions
> Drawing inspiration from popular Italian dishes, here are some unique dish names and descriptions: 1. Risotto al Limone Fresco - Creamy lemon risotto with a hint of freshness. 2. Pollo alla Cannella Croccante - Crispy cinnamon chicken with a savory twist. 3. Gnocchi al Pistacchio Affumicato - Smoked pistachio gnocchi for a unique flavor experience. 4. Salmone alla Vaniglia Piccante - Spicy vanilla salmon for a bold and aromatic dish. 5. Torta di Zucca e Caffè - Pumpkin and coffee cake, a delightful dessert fusion.

Subtask identified: Create unique dish names and descriptions base

In [25]:
# subtasks_completed should be updated with the Creative Agent's outputs
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: No task assigned
Subtasks Completed:
Subtask: Research popular Italian dishes to gather inspiration for unique dish names and descriptions
Drawing inspiration from popular Italian dishes, here are some unique dish names and descriptions: 1. Risotto al Limone Fresco - Creamy lemon risotto with a hint of freshness. 2. Pollo alla Cannella Croccante - Crispy cinnamon chicken with a savory twist. 3. Gnocchi al Pistacchio Affumicato - Smoked pistachio gnocchi for a unique flavor experience. 4. Salmone alla Vaniglia Piccante - Spicy vanilla salmon for a bold and aromatic dish. 5. Torta di Zucca e Caffè - Pumpkin and coffee cake, a delightful dessert fusion.

Subtask: Create unique dish names and descriptions based on the research
No task assigned

Is Task Compl

In [26]:
my_agent.use_agent('Chef', 'Generate ingredients for the 5 Italian dishes')

Calling function Chef with parameters {'instruction': 'Generate ingredients for the 5 Italian dishes'}

### Start of Inner Agent: Chef ###
Subtask identified: Research traditional Italian recipes for each dish and identify key ingredients
Getting LLM to perform the following task: Research traditional Italian recipes for each dish and identify key ingredients
> As Chef, I have researched traditional Italian recipes for the unique dish names provided. Here are the key ingredients for each dish: 1. Risotto al Limone Fresco - Arborio rice, lemon, chicken broth, butter, Parmesan cheese. 2. Pollo alla Cannella Croccante - Chicken, cinnamon, flour, eggs, breadcrumbs. 3. Gnocchi al Pistacchio Affumicato - Potatoes, flour, pistachios, smoked cheese, butter. 4. Salmone alla Vaniglia Piccante - Salmon, vanilla, chili flakes, olive oil, garlic. 5. Torta di Zucca e Caffè - Pumpkin, coffee, flour, sugar, eggs.

Subtask identified: Generate ingredients for the 5 Italian dishes
Getting LLM to perform

In [27]:
my_agent.use_agent('Economist', 'Generate prices for the 5 Italian dishes')

Calling function Economist with parameters {'instruction': 'Generate prices for the 5 Italian dishes'}

### Start of Inner Agent: Economist ###
Subtask identified: Generate prices for Risotto al Limone Fresco
Calling function dish_price with parameters {'list_of_dish_names': ['Risotto al Limone Fresco']}
> {'Price': ['$7']}

Subtask identified: Assign prices to the remaining 4 Italian dishes based on the unique descriptions and key ingredients.
Calling function dish_price with parameters {'list_of_dish_names': ['Pollo alla Cannella Croccante', 'Gnocchi al Pistacchio Affumicato', 'Salmone alla Vaniglia Piccante', 'Torta di Zucca e Caffè']}
> {'Price': ['$4', '$7', '$4', '$1']}

Task completed successfully!

### End of Inner Agent: Economist ###


In [28]:
my_agent.use_agent('Boss', 'Choose 3 Italian dishes to serve according to price')

Calling function Boss with parameters {'instruction': 'Choose 3 Italian dishes to serve according to price'}

### Start of Inner Agent: Boss ###
Task completed successfully!

### End of Inner Agent: Boss ###


In [29]:
# Get a targeted response from your agent, and add this to subtask_completed
my_agent.reply_user('Generate the menu items line by line in this format - Dish Name, Dish Description, Dish Ingredients, Dish Price', stateful = True)

'Risotto al Limone Fresco, Creamy lemon risotto with a hint of freshness, Arborio rice, lemon, chicken broth, butter, Parmesan cheese, $7\nPollo alla Cannella Croccante, Crispy cinnamon chicken with a savory twist, Chicken, cinnamon, flour, eggs, breadcrumbs, $4\nGnocchi al Pistacchio Affumicato, Smoked pistachio gnocchi for a unique flavor experience, Potatoes, flour, pistachios, smoked cheese, butter, $7\nSalmone alla Vaniglia Piccante, Spicy vanilla salmon for a bold and aromatic dish, Salmon, vanilla, chili flakes, olive oil, garlic, $4\nTorta di Zucca e CaffÃ¨, Pumpkin and coffee cake, a delightful dessert fusion, Pumpkin, coffee, flour, sugar, eggs, $1'

In [30]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: No task assigned
Subtasks Completed:
Subtask: Research popular Italian dishes to gather inspiration for unique dish names and descriptions
Drawing inspiration from popular Italian dishes, here are some unique dish names and descriptions: 1. Risotto al Limone Fresco - Creamy lemon risotto with a hint of freshness. 2. Pollo alla Cannella Croccante - Crispy cinnamon chicken with a savory twist. 3. Gnocchi al Pistacchio Affumicato - Smoked pistachio gnocchi for a unique flavor experience. 4. Salmone alla Vaniglia Piccante - Spicy vanilla salmon for a bold and aromatic dish. 5. Torta di Zucca e Caffè - Pumpkin and coffee cake, a delightful dessert fusion.

Subtask: Create unique dish names and descriptions based on the research
No task assigned

Subtask: Rese

In [31]:
# you can summarise the subtasks_history for the next task
my_agent.summarise_subtasks_completed('Generate the 5 Italian dishes in menu format')

In [33]:
# subtask has been summarised, helping to reduce the context significantly for later task generation
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: No task assigned
Subtasks Completed:
Subtask: Summary
Risotto al Limone Fresco, Creamy lemon risotto with a hint of freshness, Arborio rice, lemon, chicken broth, butter, Parmesan cheese, $7
Pollo alla Cannella Croccante, Crispy cinnamon chicken with a savory twist, Chicken, cinnamon, flour, eggs, breadcrumbs, $4
Gnocchi al Pistacchio Affumicato, Smoked pistachio gnocchi for a unique flavor experience, Potatoes, flour, pistachios, smoked cheese, butter, $7
Salmone alla Vaniglia Piccante, Spicy vanilla salmon for a bold and aromatic dish, Salmon, vanilla, chili flakes, olive oil, garlic, $4
Torta di Zucca e CaffÃ¨, Pumpkin and coffee cake, a delightful dessert fusion, Pumpkin, coffee, flour, sugar, eggs, $1

Is Task Completed: False
